# Training VAE

In [14]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import tensorflow as tf
import keras
from keras.layers import Input, Dense, Lambda, InputLayer, concatenate, Activation, Flatten, Reshape
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D, Deconv2D
from keras.losses import MSE
import os
from keras.models import Model, Sequential
from keras import backend as K
from keras import metrics
from keras.utils import np_utils
from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes
import utils
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
sess = tf.InteractiveSession()
K.set_session(sess)

In [17]:
batch_size=16
latent_size=8

In [18]:
# define data generators 
train_images = utils.CelebA(data_folder, sess, batch_size=batch_size)
test_images = utils.CelebA(data_folder, sess, batch_size=batch_size, train=False)
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(coord=coord)

In [19]:
vae, encoder, decoder = utils.create_vae(batch_size=batch_size, latent=latent_size)

In [25]:
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        4800      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       204800    
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 128)       512       
_________________________________________________________________
activation_2 (Activation)    (None, 16, 16, 128)       0         
__________

In [26]:
decoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 2)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 8192)              24576     
_________________________________________________________________
reshape_2 (Reshape)          (None, 4, 4, 512)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 8, 8, 512)         6553600   
_________________________________________________________________
batch_normalization_5 (Batch (None, 8, 8, 512)         2048      
_________________________________________________________________
activation_5 (Activation)    (None, 8, 8, 512)         0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 16, 16, 256)       3276800   
__________

In [24]:
vae.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (8, 64, 64, 3)       0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       multiple             4342212     input_1[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (8, 2)               0           sequential_1[1][0]               
__________________________________________________________________________________________________
lambda_2 (Lambda)               (8, 2)               0           sequential_1[1][0]               
__________________________________________________________________________________________________
lambda_3 (

In [44]:
K.set_learning_phase(True)

In [27]:
hist = vae.fit_generator(train_images,
                         steps_per_epoch=10,
                         epochs=10,
                         validation_data=test_images,
                         validation_steps=20,
                         verbose=1)

Epoch 1/10
10/10 [==============================] - 20s 2s/step - loss: 274.0180 - val_loss: 117581.3184
Epoch 2/10
10/10 [==============================] - 17s 2s/step - loss: 97.6136 - val_loss: 351957.5193
Epoch 3/10
10/10 [==============================] - 17s 2s/step - loss: 94.7726 - val_loss: 1359.6417
Epoch 4/10
10/10 [==============================] - 17s 2s/step - loss: 92.0101 - val_loss: 377.5790
Epoch 5/10
10/10 [==============================] - 17s 2s/step - loss: 85.2926 - val_loss: 156.0131
Epoch 6/10
10/10 [==============================] - 17s 2s/step - loss: 74.8609 - val_loss: 9911.3956
Epoch 7/10
10/10 [==============================] - 17s 2s/step - loss: 79.2191 - val_loss: 288.1009
Epoch 8/10
 5/10 [==============>...............] - ETA: 5s - loss: 71.8834

KeyboardInterrupt: 

In [27]:
vae.save_weights('CelebA_VAE_small_trained_localy.h5')